In [1]:
from src.modules.vin_dataclass import Vin
import src.modules.data_extractor as de
from bs4 import BeautifulSoup as BS
from pathlib import Path
from serde.json import to_json
import polars as pl

from dataclasses import dataclass
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time

import numpy as np
import re

import requests as rq

In [4]:
def scrapeur(lien : str): #"https://www.vinatis.com/achat-vin-rouge"
    ## LANCE CHROME ##
    service = Service(executable_path='./chromedriver.exe')
    driver = webdriver.Chrome(service=service)
    driver.get(lien)

    ## FERME LES COOKIES ##
    cookie_accept_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Accepter et continuer')]")
    cookie_accept_button.click()

    ## FERME POPUP ## pop-up "Inscription" apparait de manière aléatoire
    time.sleep(2)
    driver.execute_script("window.scrollBy(0, -500);")
    while True:
        try:
            time.sleep(10)
            driver.find_element(By.CLASS_NAME, 'close_modal_vinatis').click()
        except :
            continue  
        break 

    ## RECUPERE NB PAGE à SCRAPER##
    page_source = driver.page_source
    soupe = BS(page_source)
    result_nb_pages = soupe.find_all(name="span", attrs={"class": "margin-right-10"})
    NOMBRE_PAGE = int(re.findall(r'\d+', result_nb_pages[0].text)[0])

    ## BOUCLE POUR PARCOURIR LES PAGES ##
    i = 1
    vins = list()
    while i < 2:
        ## SCRAPING ##
        page_source = driver.page_source
        soupe = BS(page_source)
        
        for (item_1, item_2, item_3, item_4, item_5) in zip(
            de.extract_result(soupe)["result_title"], 
            de.extract_result(soupe)["result_price"], 
            de.extract_result(soupe)["result_avis"],
            de.extract_result(soupe)["result_type"],
            de.extract_result(soupe)["result_title_notxt"]
        ):
            vins.append(de.extract_vin(item_1, item_2, item_3, item_4, item_5))
        
        ## CHANGEMENT DE PAGE ##
        time.sleep(2)
        bouton = driver.find_element(By.CLASS_NAME, 'icon-chevron-right')
        time.sleep(2)
        driver.execute_script("arguments[0].scrollIntoView();", bouton)
        time.sleep(2)
        driver.execute_script("window.scrollBy(0, -500);")
        time.sleep(2)
        bouton.click()
        time.sleep(2)
        i = i + 1

    ## FERME CHROME ##
    driver.quit()
    
    return vins


In [5]:
vins = scrapeur("https://www.vinatis.com/achat-vin-rouge")

# RECUP ELEMENT PAGE

In [4]:
def infos(text: str):
    match = re.findall(r'[^/]+', text)
    return match

In [5]:
def extract_infos(vins: Vin):
    adjectif = list()
    type = list()
    vol = list()

    for i in range(len(vins)):
        requete = rq.get(url=vins[i].lien)
        soupe = BS(requete.text)
        
        carac = soupe.find_all(name="span", 
                attrs={"class": "ss-titre color-gray-darker taille-xs line-height-15-xs no-padding-horizontal"}
                )[0].text
        type.append(infos(carac)[0])
        
        if infos(carac)[-1].endswith("vol"):
            vol.append(infos(carac)[-1])
        else:
            vol.append(infos(carac)[-2])
        
        adjectif.append(soupe.find_all(name="div", attrs={"class": "margin-top-5 hide-xs"})[0].text)
        
    return {
        "adjectif" : adjectif,
        "type" : type,
        "vol" : vol
        }

In [22]:
def actualisation_vin(vins : Vin):
    extraction = extract_infos(vins)
    for i in range(len(vins)):
        vins[i].vol = extraction["vol"][i]
        vins[i].adjectifs = extraction["adjectif"][i]
    return vins

In [23]:
actualisation_vin(vins)

[Vin(name='GIGONDAS - LE VIEUX CLOCHER 2019 - MAISON ARNOUX & FILS', capacity=0.75, year=2019, price=216.7, promo='-20%', prix_promo=None, note=16.8, nb_avis=51, type='VIN ROUGE', lien='https://www.vinatis.com/52548-gigondas-le-vieux-clocher-2019-maison-arnoux-fils', vol=' 15 % vol', adjectifs=' Structure Souplesse Harmonie'),
 Vin(name="PETIT PONT ROUGE RESERVE - PAYS D'OC 2022 -  DOMAINES ROBERT VIC", capacity=0.75, year=2022, price=34.99, promo='-23%', prix_promo=None, note=16.1, nb_avis=13, type='VIN ROUGE', lien='https://www.vinatis.com/61066-petit-pont-rouge-reserve-pays-d-oc-2022-domaines-robert-vic', vol=' 12,5 % vol', adjectifs=' FRUITE JUTEUX PRIX/PLAISIR'),
 Vin(name='LIRAC 2019 - XAVIER VIGNON', capacity=0.75, year=2019, price=310.4, promo='-20%', prix_promo=None, note=16.2, nb_avis=154, type='VIN ROUGE', lien='https://www.vinatis.com/42740-lirac-2019-xavier-vignon', vol=' 15 % vol', adjectifs=' Gourmand Fruité Prix plaisir'),
 Vin(name='VILLA DES ANGES RESERVE 2021 - JEFF 

# JSON

In [24]:
vins_json = to_json(vins)

In [25]:
file_path = "data/vins.json"

with open(file_path, "w",  encoding='utf-8') as json_file:
    json_file.write(vins_json)

In [26]:
pl.read_json(file_path)

name,capacity,year,price,promo,prix_promo,note,nb_avis,type,lien,vol,adjectifs
str,f64,i64,f64,str,f64,f64,i64,str,str,str,str
"""GIGONDAS - LE …",0.75,2019,216.7,"""-20%""",null,16.8,51,"""VIN ROUGE""","""https://www.vi…",""" 15 % vol""",""" Structure Sou…"
"""PETIT PONT ROU…",0.75,2022,34.99,"""-23%""",null,16.1,13,"""VIN ROUGE""","""https://www.vi…",""" 12,5 % vol""",""" FRUITE JUTEUX…"
"""LIRAC 2019 - X…",0.75,2019,310.4,"""-20%""",null,16.2,154,"""VIN ROUGE""","""https://www.vi…",""" 15 % vol""",""" Gourmand Frui…"
"""VILLA DES ANGE…",0.75,2021,66.6,"""-30%""",null,16.6,148,"""VIN ROUGE""","""https://www.vi…",""" 14 % vol""",""" BOISE EQUILIB…"
"""LES DARONS 202…",0.75,2022,126.8,"""-25%""",null,16.8,15,"""VIN ROUGE""","""https://www.vi…",""" 14 % vol""",""" Gourmand Meil…"
"""LE TRIPORTEUR …",0.75,2020,34.95,"""-23%""",null,15.6,97,"""VIN ROUGE""","""https://www.vi…",""" 14,5 % vol""",""" Gourmand Orig…"
"""LUPO MERAVIGLI…",0.75,2022,38.9,"""-25%""",null,16.9,20,"""VIN ROUGE""","""https://www.vi…",""" 14,5 % vol""",""" Fruité Intens…"
"""COTES DU RHONE…",0.75,2019,36.3,"""-21%""",null,15.7,145,"""VIN ROUGE""","""https://www.vi…",""" 14 % vol""",""" Fruité Velour…"
"""COTES DU RHONE…",0.75,2020,69.9,"""-16%""",null,16.9,9,"""VIN ROUGE""","""https://www.vi…",""" 15 % vol""",""" Incontournabl…"
